# **AutoDiff**
Automatic differentiation is useful for implementing machine learning algorithms such as backpropagation for training neural networks.

##### **Setup**

In [13]:
import tensorflow as tf
import numpy as np
import matplotlib.pyplot as pl

##### **Computing Gradients**

##### **Gradient Tapes**

In [24]:
# Define a function for comoutation
def fn(x_ten):
    return x_ten**2

x = tf.Variable(3.0)

with tf.GradientTape() as tape:
    # Use defined function to transform the input.
    y = fn(x)

# Use GradientTape() to compute gradient... 
tape.gradient(y, x).numpy()

6.0

In [ ]:
### Gradients with nd-Variables
v1 = tf.Variable(tf.random.normal([3,3], 3, 9,seed= 3, name= 'var1'))
v2 = tf.Variable(tf.eye(3), name='var2')

w = [[1. ,2. ,3. ]]

# Recording operations onto `tape`
with tf.GradientTape() as tape:
    y = v1 * w + v2
    loss = tf.reduce_mean(y**2)
    
# Computing gradients w.r.t...


> **Using a dictionary of Variables with the same tape as above**

In [ ]:
(dl_dv1, dl_dv2) = tape.gradient(loss, [v1, v2])
  
# np.argsort organizes the index bsaed on the sorted order of the tensor.
np.argsort(tf.nn.softmax(dl_dv1.numpy()))
dl_dv2.numpy()

np.argsort(tf.nn.softmax(grad['dl_db']).numpy())


In [42]:
my_vars = {
    'dl_dw': v1,
    'dl_db': v2
}

grad = tape.gradient(loss, my_vars)

for i in grad:
    print(f'The name of the tensor-Variable is: {i}')
    print(grad[i].numpy())
    print()

# We discovered a new problem here...  
# If this problem occuers 
# 'A non-persistent GradientTape can only be used to compute one set of gradients (or jacobians)'
# It means the tape has previously been used to compute the gradients, thus it can not be reused.




{'dl_dw': <tf.Tensor: shape=(3, 3), dtype=float32, numpy=
array([[  0.5811825 , -10.723121  ,   6.628027  ],
       [  2.0538814 ,   8.9651165 ,  38.633232  ],
       [  3.0487926 ,   0.34223557, -26.612106  ]], dtype=float32)>, 'dl_db': <tf.Tensor: shape=(3, 3), dtype=float32, numpy=
array([[ 0.5811825 , -5.3615603 ,  2.2093422 ],
       [ 2.0538814 ,  4.4825583 , 12.877744  ],
       [ 3.0487926 ,  0.17111778, -8.870702  ]], dtype=float32)>}


<tf.Tensor: shape=(3, 3), dtype=float32, numpy=
array([[ 0.5811825 , -5.3615603 ,  2.2093422 ],
       [ 2.0538814 ,  4.4825583 , 12.877744  ],
       [ 3.0487926 ,  0.17111778, -8.870702  ]], dtype=float32)>

##### **Model Gradients**

##### **Controlling what the tape watches**

##### **Intermediate Results**

##### **Note on Performance**

##### **Gradients of non-scalar targets**

##### **Control Flow**

##### **Getting Gradient of None**

##### **No Gradient registered**

##### **Zeros instead of None**